# Image rotator

## This file constists of several steps to solve task of finding and rotating images in the right way

### Image rotator description:

In [ ]:
import pandas as pd #confortable viewing of csv tables
import numpy as np #work with arrays
import cv2 #importing images
import os #work with folders 
import matplotlib.pyplot as plt #show image
from tqdm import tqdm #progress bar

### Import data, creating samples ---> https://docs.google.com/document/d/1OvSmoV72Yfnn36p74JobDxlmbEb5OoEshQLqgFBwKPw/edit?usp=sharing

In [ ]:
data = pd.read_csv('train.truth.csv') #importing file

In [ ]:
print(len(data))
data.head() #verifying groundtruth

In [ ]:
#creating samples of different categories in different folders using os

# import os

# for index, row in data.iterrows():
#     print(row['fn'], row['label'])
# #     
#     os.rename('/home/yurii/DeeperSystems/raw_data/'+row['fn'], 
#               '/home/yurii/DeeperSystems/data/'+row['label']+'/'+row['fn'])

In [ ]:
#creating array where we will load our images
training_data = []
categories = ['rotated_left', 'upright', 'rotated_right', 'upside_down']


def create_training_data(your_path = '/home/yurii/DeeperSystems/data/'): #check your path
    '''Input: 
    your_path: str :: path where you get training images from
    Returns: list :: list of image array and label to it
    '''
    for category in categories:  

        path = os.path.join(your_path, category)  #your path
        class_num = categories.index(category)  # get the classification  

        for img in tqdm(os.listdir(path)):
            try:
                img_array = cv2.imread(os.path.join(path,img))# ,cv2.IMREAD_GRAYSCALE)  # convert to array
                training_data.append([img_array, class_num])  # add this to our training_data
            except Exception as e:  
                pass
            
            
create_training_data()

print(len(training_data))

In [ ]:
import random

random.shuffle(training_data) #for training we have to shuffle data

In [ ]:
X = []
y = []

#Separating features and labels into X, y
for features,label in training_data:
    X.append(features)
    
    y.append(label)

print(X[0].reshape(-1, 64, 64, 3))

X = np.array(X).reshape(-1, 64, 64, 3)

### Saving and loading our data using pickle

In [ ]:
# import pickle

# pickle_out = open("X.pickle","wb")
# pickle.dump(X, pickle_out)
# pickle_out.close()

# pickle_out = open("y.pickle","wb")
# pickle.dump(y, pickle_out)
# pickle_out.close()

In [ ]:
# import pickle

# pickle_in = open("X.pickle","rb")
# X = pickle.load(pickle_in)

# pickle_in = open("y.pickle","rb")
# y = pickle.load(pickle_in)


### Splitting data into train and test sets

In [ ]:
from sklearn.model_selection import train_test_split

#Using test_size=0.33 to split data saved before
X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33)

### importing keras and initializing model

In [ ]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import SGD
from keras.models import load_model

np.random.seed(42)

In [ ]:
batch_size = 32
# nb of  classes ('rotated_left', 'upright', 'rotated_right', 'upside_down')
nb_classes = 4
# number of epochs
nb_epoch = 10
# image size
img_rows, img_cols = 64, 64
# We didn`nt convert our image into grayscale so we have RGB
img_channels = 3


# Data normalization. better do here (after test_train_split) of MEMORY ERROR!!!!!!
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# labels to categories
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

# Model creation
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                        input_shape=(64, 64, 3), activation='relu'))

model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))

#optimization parameters
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
# Training 
model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=nb_epoch,
              validation_split=0.1,
              shuffle=True,
              verbose=2)



In [ ]:
# Evaluating our model on test data
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Accuracy on test data: %.2f%%" % (scores[1]*100))

### Saving our model

In [ ]:
model.save('image_rotator_net.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model

In [ ]:
# returns a compiled model
# identical to the previous one
model = load_model('image_rotator_net.h5')

### Loading test data

In [ ]:
#test data
testing_data = []
path = '/home/yurii/DeeperSystems/test'
test_files = os.listdir(path)
for img in tqdm(test_files): 
    try:
        img_array = cv2.imread(os.path.join(path,img))
        testing_data.append(img_array)  # add this to our training_data
    except Exception as e:  
        pass

In [ ]:
#also normilizing. its better to use NumPy here to normilize data bewaring of memory error
testing_data_n = np.divide(testing_data, 255)

### Predictions on test data

In [ ]:
predictions = model.predict(testing_data_n)
predictions = [np.argmax(i) for i in predictions]

In [ ]:
test_output = pd.DataFrame(list(zip(test_files, predictions)), 
               columns =['fn', 'label']) 
test_output['label'] = test_output['label'].map({
        0:'rotated_left', 
        1:'upright', 
        2:'rotated_right', 
        3:'upside_down'
    })

In [ ]:
test_output.head(5)

In [ ]:
test_output.to_csv('test.preds.csv') #saving ourout

### Rotating images

In [ ]:
#using funtion from eval.py to create dictionary
def load_csv(fn='test.preds.csv'):
    res = {}
    import csv
    for row in csv.DictReader(open(fn)):
        res[row['fn']] = row['label']
    return res


In [ ]:
im_to_rotate = load_csv() #initializing dictionary

In [ ]:
def rotate_image(d, image):
    '''rotate_image:
    Input:
    d: dict {'fn':'label'} :: consists of labeled images
    image: str :: name of image which is in d
    Returns: list :: rotated image array'''
    
    angle = 0
    
    if d[image] == 'rotated_right':
        angle = 90

    if d[image] == 'rotated_left':
        angle = 270

    if d[image] == 'upside_down':
        angle = 180
        
    image = cv2.imread(os.path.join(path,image))
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result


In [ ]:
#saving to PNG!
for i in im_to_rotate:
    cv2.imwrite('/home/yurii/DeeperSystems/rotated_images/'+i[:-4]+'.png', rotate_image(im_to_rotate, i))

### Creating numpy output

In [ ]:
#using already known code, reading image
numpy_output_data = []
path = '/home/yurii/DeeperSystems/rotated_images/'
test_files = os.listdir(path)
for img in tqdm(test_files): 
    try:
        img_array = cv2.imread(os.path.join(path,img))
        numpy_output_data.append(img_array)  # add this to our training_data
    except Exception as e:  
        pass

In [ ]:
numpy_output = np.array(numpy_output_data).reshape(-1, 64, 64, 3)

In [ ]:
numpy_output.shape

In [ ]:
np.save('numpy_output.out', numpy_output)